In [1]:
# import những thư viện cần thiết
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV


In [2]:
# Đọc dữ liệu từ file csv
df = pd.read_csv('../data/cleaned_dataset.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1667 entries, 0 to 1666
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Khu vực tuyển         1667 non-null   object 
 1   Thời gian thử việc    1667 non-null   float64
 2   Cấp bậc               1667 non-null   object 
 3   Yêu cầu giới tính     1667 non-null   object 
 4   Số lượng tuyển        1667 non-null   int64  
 5   Hình thức làm việc    1667 non-null   object 
 6   Yêu cầu bằng cấp      1667 non-null   object 
 7   Yêu cầu kinh nghiệm   1667 non-null   object 
 8   Ngành nghề            1667 non-null   object 
 9   Quy mô công ty        1667 non-null   object 
 10  Loại công ty          1667 non-null   int64  
 11  Mức lương thấp nhất   1667 non-null   float64
 12  Mức lương cao nhất    1667 non-null   float64
 13  Mức lương trung bình  1667 non-null   float64
 14  Tuổi thấp nhất        1667 non-null   float64
 15  Tuổi cao nhất        

In [27]:
mutil_label_features = ['Khu vực tuyển', 'Ngành nghề']
single_label_features = [ 'Thời gian thử việc', 'Cấp bậc', 'Số lượng tuyển','Hình thức làm việc','Yêu cầu bằng cấp', 
                'Yêu cầu kinh nghiệm', 'Quy mô công ty', 'Loại công ty',  'Tuổi thấp nhất', 'Tuổi cao nhất']
targets = ['Mức lương trung bình']


# Xử lý các cột có nhiều nhãn
df_one_hot_encoded = pd.get_dummies(df[single_label_features], columns=single_label_features)

# Chức năng để tách dữ liệu
def split_labels(column, separator):
    return [[label.strip() for label in labels.split(separator)] for labels in column]

# Tách 'Khu vực tuyển'
khu_vuc_labels = split_labels(df['Khu vực tuyển'], ', ')

# Tách 'Ngành nghề'
nganh_nghe_labels = split_labels(df['Ngành nghề'], '/')

# Khởi tạo và áp dụng MultiLabelBinarizer
mlb_khu_vuc = MultiLabelBinarizer()
mlb_nganh_nghe = MultiLabelBinarizer()

khu_vuc_binarized = mlb_khu_vuc.fit_transform(khu_vuc_labels)
nganh_nghe_binarized = mlb_nganh_nghe.fit_transform(nganh_nghe_labels)

# Tạo DataFrame mới từ kết quả nhị phân
khu_vuc_df = pd.DataFrame(khu_vuc_binarized, columns=mlb_khu_vuc.classes_)
nganh_nghe_df = pd.DataFrame(nganh_nghe_binarized, columns=mlb_nganh_nghe.classes_)

# Đặt lại chỉ mục cho các DataFrame
df_one_hot_encoded.reset_index(drop=True, inplace=True)
khu_vuc_df.reset_index(drop=True, inplace=True)
nganh_nghe_df.reset_index(drop=True, inplace=True)

df_one_hot_encoded = pd.concat([df_one_hot_encoded, khu_vuc_df, nganh_nghe_df], axis=1).astype('int64')

    
df_one_hot_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1667 entries, 0 to 1666
Columns: 227 entries, Thời gian thử việc_0.0 to Điện - Điện tử - Điện lạnh
dtypes: int64(227)
memory usage: 2.9 MB


In [28]:
df_one_hot_encoded

,Thời gian thử việc_0.0,Thời gian thử việc_1.0,Thời gian thử việc_2.0,Thời gian thử việc_3.0,Thời gian thử việc_6.0,Cấp bậc_Chuyên gia,Cấp bậc_Chuyên viên- nhân viên,Cấp bậc_Cộng tác viên,Cấp bậc_Quản lý cấp cao,Cấp bậc_Quản lý cấp trung,...,Thực tập sinh,Truyền hình - Báo chí - Biên tập,Tài chính - Đầu tư - Chứng Khoán,Vận Tải - Lái xe - Giao nhận,Vận hành - Bảo trì - Bảo dưỡng,Xuất Nhập Khẩu,Xuất bản - In ấn,Xây dựng,Y tế - Chăm sóc sức khỏe,Điện - Điện tử - Điện lạnh
0,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1662,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1663,0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1664,0,1,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1665,0,0,1,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1


In [ ]:
# Chuẩn bị dữ liệu (thay đổi tên cột và đường dẫn dữ liệu tương ứng)
X = df_one_hot_encoded
y = df['Mức lương trung bình']

# Chia dữ liệu
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Xây dựng và huấn luyện mô hình
models = {
    'Random Forest': RandomForestRegressor(),
    'Gradient Boosting': GradientBoostingRegressor(),
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'K-Nearest Neighbors': KNeighborsRegressor(),
    'Decision Tree': DecisionTreeRegressor(),
    'AdaBoost': AdaBoostRegressor(),
    'Extra Trees': ExtraTreesRegressor(),
    'XGBoost': XGBRegressor(),
    'LightGBM': LGBMRegressor()
}

# Thiết lập các tham số cần tìm kiếm
param_grid = {
    'Random Forest': {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20], 'min_samples_split': [2, 5, 10]},
    'Gradient Boosting': {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 0.2], 'max_depth': [3, 5, 7]},
    'Linear Regression': {},
    'Ridge Regression': {'alpha': [0.1, 1, 10], 'solver': ['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']},
    'Lasso Regression': {'alpha': [0.1, 1, 10], 'tol': [0.001, 0.01, 0.1]},
    'K-Nearest Neighbors': {'n_neighbors': [3, 5, 7], 'weights': ['uniform', 'distance'], 'algorithm': ['ball_tree', 'kd_tree', 'brute']},
    'Decision Tree': {'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]},
    'AdaBoost': {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 0.2]},
    'Extra Trees': {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]},
    'XGBoost': {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 0.2], 'max_depth': [3, 5, 7], 'min_child_weight': [1, 3, 5]},
    'LightGBM': {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 0.2], 'max_depth': [3, 5, 7], 'min_child_samples': [5, 10, 20]}
}


# K-fold Cross Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Grid Search và huấn luyện mô hình
best_models = {}

# Kích thước của ô vẽ
fig, axs = plt.subplots(nrows=3, ncols=4, figsize=(16, 16))
axs = axs.flatten()

best_params = []
mses = []
r2s = []
model_names = []

# Grid Search và huấn luyện mô hình
for i, (model_name, model) in enumerate(models.items()):
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid[model_name], scoring='neg_mean_squared_error', cv=kf)
    grid_search.fit(X_train, y_train)

    best_models[model_name] = grid_search.best_estimator_

    predictions = grid_search.best_estimator_.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)

    model_names.append(model_name)
    best_params.append(grid_search.best_params_)
    mses.append(mse)
    r2s.append(r2)

    # Vẽ biểu đồ so sánh giữa giá trị thực tế và dự đoán
    sns.scatterplot(x=y_test, y=predictions, ax=axs[i])
    axs[i].set_title(f'{model_name} - Actual vs Predicted')
    axs[i].set_xlabel('Actual Salary')
    axs[i].set_ylabel('Predicted Salary')

# Tinh chỉnh layout
plt.tight_layout()
plt.show()

In [ ]:
for i, (model_name, model) in enumerate(models.items()):
  print('{} có mean square erros{} & r square erro: {}'.format(model_name,mses[i],r2s[i]))